In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms

# Load Pretrained CNN Model (Can replace Inception-V3 with ResNet, CLIP, etc.)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = models.inception_v3(pretrained=True).to(device)  # Change this for a different model
model.eval()

# Define Image Transform for Feature Extraction (Change size if using a different model)
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Adjust size for different models (224x224 for ResNet, etc.)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
# Function to Extract Features from an Image Segment
def extract_features(image):
    image = transform(image).unsqueeze(0).to(device)  # Transform and add batch dimension
    with torch.no_grad():
        features = model(image).cpu().numpy().flatten()  # Extract features (adjust for CLIP)
    return features


In [ ]:
import cv2
import numpy as np
from skimage.segmentation import slic
from skimage.color import rgb2lab
from PIL import Image

# Function to Perform Multi-Resolution Segmentation
def segment_image(image_path, num_segments_list=[50, 100, 150]):  # Change number of segments
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for PIL compatibility
    segments_list = []

    for num_segments in num_segments_list:
        segments = slic(image, n_segments=num_segments, compactness=10, sigma=1)  # Tune params
        segments_list.append(segments)

    return image, segments_list


In [ ]:
from sklearn.cluster import KMeans

# Function to Cluster Similar Image Segments
def cluster_segments(image_path, num_clusters=10):  # Adjust number of clusters
    image, segments_list = segment_image(image_path)
    feature_vectors = []

    for segments in segments_list:
        unique_segments = np.unique(segments)
        for seg_id in unique_segments:
            mask = segments == seg_id
            segment_img = np.zeros_like(image)
            segment_img[mask] = image[mask]
            pil_img = Image.fromarray(segment_img)
            feature_vector = extract_features(pil_img)
            feature_vectors.append(feature_vector)

    # Perform Clustering on Extracted Features
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)  # Change clustering algorithm
    cluster_labels = kmeans.fit_predict(feature_vectors)

    return cluster_labels, feature_vectors


In [ ]:
import os
from collections import Counter

# Function to Extract Concepts from a Set of Images
def extract_custom_concepts(image_folder):
    all_clusters = []

    for image_file in os.listdir(image_folder):
        if image_file.endswith((".jpg", ".png", ".jpeg")):
            image_path = os.path.join(image_folder, image_file)
            cluster_labels, _ = cluster_segments(image_path)
            all_clusters.extend(cluster_labels)

    # Count the Frequency of Each Cluster (Custom Concepts)
    concept_counts = Counter(all_clusters)
    
    # Sort and Display the Most Common Concepts
    sorted_concepts = sorted(concept_counts.items(), key=lambda x: x[1], reverse=True)
    print("Extracted Concepts (Clustered Patterns):")
    for concept, count in sorted_concepts:
        print(f"Concept {concept}: {count} occurrences")

    return sorted_concepts


In [ ]:
# Automatic Concept Extraction on a Folder of Images
image_folder = "path_to_images"
concepts = extract_custom_concepts(image_folder)

In [ ]:
import matplotlib.pyplot as plt
import random

# Function to Visualize Image Segments in Clusters
def visualize_clusters(image_path, num_clusters=5, samples_per_cluster=4):
    image, segments_list = segment_image(image_path)
    cluster_labels, feature_vectors = cluster_segments(image_path, num_clusters=num_clusters)

    fig, axes = plt.subplots(num_clusters, samples_per_cluster, figsize=(10, 2 * num_clusters))
    
    cluster_to_segments = {i: [] for i in range(num_clusters)}

    # Group segments by cluster
    for seg_id, cluster in enumerate(cluster_labels):
        cluster_to_segments[cluster].append(seg_id)

    # Plot samples from each cluster
    for cluster_idx, seg_ids in cluster_to_segments.items():
        sampled_segments = random.sample(seg_ids, min(samples_per_cluster, len(seg_ids)))

        for j, seg_id in enumerate(sampled_segments):
            mask = np.zeros_like(image)
            for segments in segments_list:
                mask[segments == seg_id] = image[segments == seg_id]

            axes[cluster_idx, j].imshow(mask)
            axes[cluster_idx, j].axis("off")

    plt.suptitle(f"Visualization of {num_clusters} Concept Clusters", fontsize=14)
    plt.show()
